In [ ]:
import sqlite3
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel
from dotenv import load_dotenv
import os
import json
import random

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

search_tool = DuckDuckGoSearchTool()

# Define the agent
model = LiteLLMModel(
    "llama-3.3-70b-versatile",
    api_base="https://api.groq.com/openai/v1",
    api_key=api_key
)

# model = LiteLLMModel(
#     model_id="anthropic/claude-3-5-sonnet-20240620",
#     # api_base="https://api.anthropic.com/v1",
#     api_key=api_key
# )
model.flatten_messages_as_text = True

agent = CodeAgent(
    tools=[search_tool],
    model=model,
    add_base_tools=False,
    verbosity_level=2
)


# Define the function to choose the best algorithm
def choose_best_algorithm(train_db_file="../train_mTSP.sqlite3", test_db_file="../test_mTSP.sqlite3"):

    conn_train = sqlite3.connect(train_db_file)
    cursor_train = conn_train.cursor()

    # Select instances from the training database
    cursor_train.execute("""
        SELECT instance_id, nr_cities, nr_salesmen, average_distance, stddev_distance, density, salesmen_ratio, 
               bounding_box_area, aspect_ratio, spread, cluster_compactness, mst_total_length, entropy_distance_matrix 
        FROM instances 
    """)
    train_instances = cursor_train.fetchall()
    train_instances_with_columns = [
        {
            "instance_id": row[0], 
            "nr_cities": row[1], 
            "nr_salesmen": row[2],
            "average_distance": row[3],
            "stddev_distance": row[4],
            "density": row[5],
            "salesmen_ratio": row[6],
            "bounding_box_area": row[7],
            "aspect_ratio": row[8],
            "spread": row[9],
            "cluster_compactness": row[10],
            "mst_total_length": row[11],
            "entropy_distance_matrix": row[12]
        } for row in train_instances
    ]

    # Select algorithms from the training database
    cursor_train.execute("""
        SELECT instance_id, strategy, total_cost, normalized_cost, time_taken, distance_gap, efficiency 
        FROM algorithms 
        WHERE instance_id IN (SELECT instance_id FROM instances)
    """)
    train_algorithms_data = cursor_train.fetchall()
    train_algorithms_with_columns = [
        {
            "instance_id": row[0],
            "strategy": row[1],
            "total_cost": row[2],
            "normalized_cost": row[3],
            "time_taken": row[4],
            "distance_gap": row[5],
            "efficiency": row[6],
        }
        for row in train_algorithms_data
    ]


    conn_train.close()
    conn_test = sqlite3.connect(test_db_file)
    cursor_test = conn_test.cursor()

    # Select instances from the test database
    cursor_test.execute("""
        SELECT instance_id, nr_cities, nr_salesmen, average_distance, stddev_distance, density, salesmen_ratio, 
               bounding_box_area, aspect_ratio, spread, cluster_compactness, mst_total_length, entropy_distance_matrix 
        FROM instances 
    """)
    test_instances = cursor_test.fetchall()
    test_instances_with_columns = [
        {
            "instance_id": row[0], 
            "nr_cities": row[1], 
            "nr_salesmen": row[2],
            "average_distance": row[3],
            "stddev_distance": row[4],
            "density": row[5],
            "salesmen_ratio": row[6],
            "bounding_box_area": row[7],
            "aspect_ratio": row[8],
            "spread": row[9],
            "cluster_compactness": row[10],
            "mst_total_length": row[11],
            "entropy_distance_matrix": row[12]
        } for row in test_instances
    ]

    # test_instances_with_columns = test_instances_with_columns[:5]


    conn_test.close()

    # sample a subset of training instances
    nr_train = len(train_instances_with_columns)
    sample_size = max(1, nr_train // 20)
    sampled_instances = random.sample(train_instances_with_columns, sample_size)
    sampled_ids = set(inst["instance_id"] for inst in sampled_instances)

    # filter algorithms to only include those that match the sampled instances
    sampled_algorithms = [alg for alg in train_algorithms_with_columns if alg["instance_id"] in sampled_ids]


    # Prepare the data for the agent
    data = {
        "train_instances": sampled_instances,
        "train_algorithms": sampled_algorithms,
        "test_instances": test_instances_with_columns,
    }
    data_str = json.dumps(data, indent=4)

    # {data_str}

    # Task for the agent
    task = f"""
    You are an AI agent tasked with selecting the best algorithm for solving each instance of the mTSP problem.
    Below is the data from the training instances and their results for different strategies:

    {data_str}

    To evaluate each strategy, use the following composite score formula (lower is better):

    1. For each instance, normalize 'time_taken' and 'distance_gap' among all strategies for that instance.
    2. Compute a bonus/penalty for each strategy:
       - Greedy: -0.02
       - KMeans Greedy: 0.0
       - Branch and Cut: +0.2 if nr_cities > 40, else -0.3
       - Ant Colony: -0.1
    3. The composite score is:
       composite_score = normalized_cost * 0.65 + time_taken_norm * 0.25 + distance_gap_norm * 0.1 + strat_bonus

    Your task is to:
    1. Analyze the performance of each strategy (Greedy, Branch and Cut, Ant Colony, KMeans-Greedy) on the training instances.
    2. Based on this analysis, recommend the best strategy for each test instance.
    3. Justify your recommendations based on the training data.
    4. Consider the following performance metrics: total cost, normalized_cost, time taken, efficiency, and distance gap. The total cost is the most important metric, followed by normalized_cost and efficiency. Time taken is less important, but it should be considered for strategies that are computationally expensive.
    5. Take into account that Branch and Cut and Ant Colony are more computationally expensive, while Greedy and KMeans-Greedy are faster but may not always yield the best results.
    6. Branch and Cut and Ant Colony have a 20 seconds time limit, so they may output a weaker solution if the time limit is reached or even no solution at all.

    Output your recommendations in the following format as in a CSV file:
    instance_id,predicted_algorithm
    where:
    - instance_id is the ID of the test instance
    - predicted_algorithm is the algorithm you recommend for that instance 
    """

    response = agent.run(task)
    print(response)

choose_best_algorithm(train_db_file="../train_mTSP.sqlite3", test_db_file="../test_mTSP.sqlite3")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an AI agent tasked with selecting the best algorithm for solving each instance of the mTSP problem.     │
│     Below is the data from the training instances and their results for different strategies:                   │
│                                                                                                                 │
│     {                                                                                                           │
│     "train_instances": [                                                                                        │
│         {                                                                                                       │
│             "instance_id": 733,                                                                                 │
│             "nr_cities": 70,                                                                                    │
│             "nr_salesmen": 3,                                                                                   │
│             "average_distance": 52.02767378185998,                                                              │
│             "stddev_distance": 24.62586104232115,                                                               │
│             "density": 0.0052553205840262605,                                                                   │
│             "salesmen_ratio": 0.04285714285714286,                                                              │
│             "bounding_box_area": 9900.0,                                                                        │
│             "aspect_ratio": 1.0101010101010102,                                                                 │
│             "spread": 2.5020684836852327,                                                                       │
│             "cluster_compactness": 0.7432524825979997,                                                          │
│             "mst_total_length": 627.0455515523668,                                                              │
│             "entropy_distance_matrix": 2.726876830726254                                                        │
│         },                                                                                                      │
│         {                                                                                                       │
│             "instance_id": 313,                                                                                 │
│             "nr_cities": 30,                                                                                    │
│             "nr_salesmen": 3,                                                                                   │
│             "average_distance": 6.201474309196723,                                                              │
│             "stddev_distance": 2.870277092184906,                                                               │
│             "density": 0.06201474309196722,                                                                     │
│             "salesmen_ratio": 0.1,                                                                              │
│             "bounding_box_area": 100.0,                                                                         │
│             "aspect_ratio": 1.0,                                                                                │
│             "spread": 2.169423491301435,                                                                        │
│             "cluster_compactness": 0.20671581030655742,                                                         │
│             "mst_total_length": 47.49789859063726,    

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



Error in generating model output:
litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for model 
`llama-3.3-70b-versatile` in organization `org_01jqy00a9ffc4r88wt8avqh336` service tier `on_demand` on tokens per 
minute (TPM): Limit 12000, Requested 28426, please reduce your message size and try again. Need more tokens? 
Upgrade to Dev Tier today at 
https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

[Step 1: Duration 0.62 seconds]

AgentGenerationError: Error in generating model output:
litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for model `llama-3.3-70b-versatile` in organization `org_01jqy00a9ffc4r88wt8avqh336` service tier `on_demand` on tokens per minute (TPM): Limit 12000, Requested 28426, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
